# Graph-RAG sobre el documento de AWS

Este notebook te permite interactuar con tu grafo de conocimiento (Neo4j) usando RAG (Retrieval-Augmented Generation) sobre el documento:

- "AWS Documento técnico – Descripción general de Amazon Web Services" (aprox. 170 páginas)

El enfoque está en servicios de AWS, categorías, regiones, características e integraciones.

## ¿Cómo funciona?

1. **Haces una pregunta** en lenguaje natural (en español)
2. **GPT (OpenAI)** genera automáticamente una query Cypher basada en el esquema del grafo
3. **Neo4j** ejecuta la query y retorna los resultados
4. **GPT (OpenAI)** genera una respuesta natural basada exclusivamente en los resultados

## Prerequisitos

- Neo4j corriendo con los datos extraídos del PDF de AWS
- API key de OpenAI en el archivo `.env`


## Paso 1: Importar librerías y configurar


In [14]:
import os
import importlib
from dotenv import load_dotenv

load_dotenv()

# Reiniciar el módulo para cargar cambios
import pdf_kg
importlib.reload(pdf_kg)
from pdf_kg import PDFKnowledgeGraph, format_results_as_markdown_table

print("✓ Librerías importadas")
print(f"✓ Neo4j: {os.environ.get('NEO4J_HOST', 'localhost')}:{os.environ.get('NEO4J_PORT', '7687')}")
print(f"✓ OpenAI Model: {os.environ.get('OPENAI_MODEL', 'gpt-4')}")


✓ Librerías importadas
✓ Neo4j: localhost:7687
✓ OpenAI Model: gpt-4


## Paso 2: Conectar al Knowledge Graph


In [15]:
# Inicializar el Knowledge Graph
kg = PDFKnowledgeGraph()

# Verificar conexión
try:
    test_result = kg.query_database("RETURN 1 as test")
    print("✓ Conexión a Neo4j establecida")
except Exception as e:
    print(f"✗ Error conectando a Neo4j: {e}")
    raise


✓ Conexión a Neo4j establecida


## Paso 3: Ver estadísticas del grafo

Veamos qué datos tienes en el grafo.


In [3]:
# Contar entidades del PDF
entities_query = """
MATCH (n:ENTITY) WHERE n.source = 'pdf'
RETURN count(n) as total_entities
"""
entities_result = kg.query_database(entities_query)
print(f"📊 Total de entidades del PDF: {entities_result[1][0] if len(entities_result) > 1 else 0}")

# Contar relaciones del PDF
relations_query = """
MATCH ()-[r:RELATED_TO]->() WHERE r.source = 'pdf'
RETURN count(r) as total_relations
"""
relations_result = kg.query_database(relations_query)
print(f"🔗 Total de relaciones del PDF: {relations_result[1][0] if len(relations_result) > 1 else 0}")

# Ver tipos de entidades
types_query = """
MATCH (n:ENTITY) WHERE n.source = 'pdf'
RETURN labels(n) as tipo, count(n) as cantidad
ORDER BY cantidad DESC
LIMIT 10
"""
types_result = kg.query_database(types_query)
print("\n📋 Tipos de entidades (top 10):")
for row in types_result[1:]:
    labels = [l for l in row[0] if l != 'ENTITY']
    tipo = ':'.join(labels) if labels else 'ENTITY'
    print(f"  {tipo}: {row[1]}")


PASO 3: Ejecutando query: 
MATCH (n:ENTITY) WHERE n.source = 'pdf'
RETURN count(n) as total_entities

📊 Total de entidades del PDF: 2002
PASO 3: Ejecutando query: 
MATCH ()-[r:RELATED_TO]->() WHERE r.source = 'pdf'
RETURN count(r) as total_relations

🔗 Total de relaciones del PDF: 8452
PASO 3: Ejecutando query: 
MATCH (n:ENTITY) WHERE n.source = 'pdf'
RETURN labels(n) as tipo, count(n) as cantidad
ORDER BY cantidad DESC
LIMIT 10


📋 Tipos de entidades (top 10):
  ENTITY: 1664
  ORG: 171
  LOC: 99
  PERSON: 53
  ORG:LOC: 7
  PERSON:LOC: 6
  PERSON:ORG: 2


## Paso 4: Hacer una pregunta usando RAG

Escribe tu pregunta en la celda de abajo. El sistema:
1. Generará automáticamente una query Cypher
2. Ejecutará la query en Neo4j
3. Generará una respuesta natural basada en los resultados


In [25]:
pregunta = "¿Qué servicios de almacenamiento menciona el documento?"
pregunta = "Lista los servicios relacionados con seguridad"
pregunta = "que servicios existen para depslegar una API a miles de clientes?"
pregunta = "Cuales son los tipos de relaciones que existen?"
pregunta = "necesito desplegar una aplicacion web, que servicios puedo usar?"

print(f"❓ Pregunta: {pregunta}\n")
print("=" * 60)

# 1. Generar query Cypher usando GPT
print("🤖 Generando query Cypher con GPT...")
cypher_query = kg.get_candidate_cypher_query(pregunta)
print(f"✓ Query generada:\n{cypher_query}\n")

# 2. Ejecutar query en Neo4j
print("🔍 Ejecutando query en Neo4j...")
results = kg.query_database(cypher_query)
formatted_results = format_results_as_markdown_table(results)
print(f"✓ Resultados obtenidos\n")

# 3. Mostrar resultados
print("📊 Resultados:")
print(formatted_results)
print("=" * 60)

# 4. Generar respuesta usando GPT
print("\n🤖 Generando respuesta con GPT...")
respuesta = kg.generate_response(pregunta, formatted_results)
print("\n" + "=" * 60)
print("💬 RESPUESTA:")
print("=" * 60)
print(respuesta)


❓ Pregunta: necesito desplegar una aplicacion web, que servicios puedo usar?

🤖 Generando query Cypher con GPT...
✓ Query generada:
MATCH (s:ENTITY)
WHERE toLower(s.name) CONTAINS 'web' OR toLower(s.name) CONTAINS 'app'
RETURN DISTINCT s.name AS servicio
LIMIT 50;

🔍 Ejecutando query en Neo4j...
PASO 4: Formateando resultados a Tabla Markdown
✓ Resultados obtenidos

📊 Resultados:
| servicio                                                                                                      |
|---------------------------------------------------------------------------------------------------------------|
| descripcion-general-de-amazon-web-services-copyright                                                          |
| amazon-web-services                                                                                           |
| descripcion-general-de-amazon-web-services-aws-documento                                                      |
| descripcion-general-de-amazon-web-services   

## Paso 5: Más ejemplos de preguntas

Puedes hacer diferentes tipos de preguntas. Algunos ejemplos:


In [ ]:
# Ejemplos adicionales orientados a AWS
#pregunta1 = "¿Qué servicios de base de datos menciona el documento?"
#pregunta1 = "Lista servicios relacionados con 'security' o 'IAM'"
#pregunta1 = "¿Qué servicios aparecen EN la región us-east-1?"
pregunta1 = "¿Con qué servicios se integra AWS Lambda?"

print(f"❓ {pregunta1}\n")
cypher1 = kg.get_candidate_cypher_query(pregunta1)
results1 = kg.query_database(cypher1)
print(f"✓ Resultados obtenidos\n {results1}")
respuesta1 = kg.generate_response(pregunta1, format_results_as_markdown_table(results1))
print(f"💬 {respuesta1}\n")


❓ cuantas veces se nombra gonzalo

PASO 0: Pregunta: cuantas veces se nombra gonzalo
PASO 1: Generando prompt del sistema
PASO 3: Ejecutando query: 
            CALL apoc.meta.data()
            YIELD label, other, elementType, type, property
            WHERE elementType = 'node' AND type <> 'RELATIONSHIP'
            WITH label AS nodeLabels, collect(DISTINCT property) AS properties
            RETURN {labels: nodeLabels, properties: properties} AS output
        
PASO 3: Ejecutando query: 
            CALL apoc.meta.data()
            YIELD label, other, elementType, type, property
            WHERE elementType = 'relationship'
            RETURN {source: label, relationship: property, target: other} AS output
        
PASO 3: Respuesta: MATCH (p:PERSON {name: 'gonzalo'}) RETURN COUNT(p) as count
PASO 3: Ejecutando query: MATCH (p:PERSON {name: 'gonzalo'}) RETURN COUNT(p) as count
✓ Resultados obtenidos
 [['count'], [0]]
PASO 4: Formateando resultados a Tabla Markdown
PASO 5: Genera

## Paso 6: Ver el esquema del grafo

El sistema obtiene automáticamente el esquema del grafo para generar queries correctas.


In [7]:
# Obtener y mostrar el esquema completo
system_prompt = kg.get_cypher_system_prompt()
print("📋 Esquema del grafo (usado por GPT-4 para generar queries):")
print("=" * 60)
print(system_prompt)


PASO 1: Generando prompt del sistema
PASO 3: Ejecutando query: 
            CALL apoc.meta.data()
            YIELD label, other, elementType, type, property
            WHERE elementType = 'node' AND type <> 'RELATIONSHIP'
            WITH label AS nodeLabels, collect(DISTINCT property) AS properties
            RETURN {labels: nodeLabels, properties: properties} AS output
        
PASO 3: Ejecutando query: 
            CALL apoc.meta.data()
            YIELD label, other, elementType, type, property
            WHERE elementType = 'relationship'
            RETURN {source: label, relationship: property, target: other} AS output
        
📋 Esquema del grafo (usado por GPT-4 para generar queries):
## Task:

            Generate Cypher queries to query a Neo4j graph database based on the provided schema definition.

            ## Instructions:

            You are an expert at generating Cypher queries to query a Neo4j graph database based on the provided schema definition.
           

## Paso 7: Cerrar conexión

Cuando termines, cierra la conexión a Neo4j.


In [8]:
kg.close()
print("✓ Conexión cerrada")


✓ Conexión cerrada
